Importação e Leitura dos Dados

In [ ]:
import pandas as pd
from datetime import datetime
import os
import sys
import calendar

In [ ]:
df = pd.read_csv('../data/raw/superstore_final_dataset.csv', encoding='latin1')

Entendendo a base de Dados

Nessa sessão será executado alguns comandos, para entender melhor sobre a estrutura e como funciona o dataset

In [ ]:
df.head(10)

In [ ]:
df.info() # Entendendo melhor como funciona e está estruturado os dados

In [ ]:
df.columns # Verificando quais são as colunas existentes nesse dataset

In [ ]:
df.describe()

Preparação dos Dados

Nessa sessão será executado as principais formatações, limpezas e preparações dos dados para as análise. Garantindo a consistência e clareza dos dados

In [ ]:
df_copy = df.copy()
df_copy = df_copy.set_index('Row_ID')

In [ ]:
df_copy.isnull().sum()
mask_index_nan = df_copy['Postal_Code'].isnull() 
# Uma máscara com os index onde o Postal_Code está com o valor NaN, para ter um controle exato de onde essses valores estão

In [ ]:
# Mudar os valores NaN do Postal_Code para o valor 0
df_copy.loc[df_copy['Postal_Code'].isnull(), 'Postal_Code'] = 0
df_copy[mask_index_nan] # Verificar se a correção foi feita corretamente, se sim não deve retornar nenhuma linha

In [ ]:
df_copy.loc[df_copy.duplicated()]
# Ter o controle e visualização de qual linha realmente é a duplicada, somente para uma breve visualização das linhas
linhas_duplicadas = df_copy['Order_ID'] == 'US-2015-150119'

In [ ]:
# Remover as duplicatas
mask = ~df_copy.duplicated(keep='first') # Uso do operador not, para que as duplicatas sejam false, e ao aplicar a máscara no df, ela não apareça, pois é false
df_copy = df_copy[mask].copy()

In [ ]:
# Função para formatar as datas
def formatar_datas(valor):
    date = valor.split('/')
    nova_data = f'{date[2]}-{date[1]}-{date[0]}'
    return nova_data


In [ ]:
df_copy['Order_Date'] = df_copy['Order_Date'].apply(formatar_datas)
df_copy['Ship_Date'] = df_copy['Ship_Date'].apply(formatar_datas)

In [ ]:
# Converter Order_Date e Ship_Date para o tipo datetime
df_copy['Order_Date'] = pd.to_datetime(df_copy['Order_Date'], errors='coerce')
df_copy['Ship_Date'] = pd.to_datetime(df_copy['Ship_Date'], errors='coerce')

In [ ]:
df_copy['Sales'] = df_copy['Sales'].map("{:.2f}".format)
df_copy['Sales'] = pd.to_numeric(df_copy['Sales'], errors='coerce')

Analisar o total de vendas entre 2015-2019

Nessa sessão será extraído o valor total de arrecadação do supermercado nos anos de 2015 a 2019

In [ ]:
total_vendas = df_copy['Sales'].sum()
total_vendas

Analisar o total de vendas por ano entre 2015-2019

Nessa sessão será extraído a arrecadação do supermercado em cada ano individual

In [ ]:
# Separar cada ano
vendas_por_ano = {} # Local onde a quantidade de vendas de cada ano será armazenado
def soma_vendas_por_ano(valor):
    for x in range(5, 10):
        ano = int(f'201{x}')
        query = round(df_copy.loc[df_copy['Ship_Date'].dt.year == ano, 'Sales'].sum(), 2)
        valor.update({f'201{x}':float(query)})
    series = pd.Series(valor)
    return pd.DataFrame(series, columns=['Total de Vendas']).sort_values(by='Total de Vendas', ascending=False)
soma_vendas_por_ano(vendas_por_ano)

Identificação Mensal dos Picos e Quedas de Vendas Durante o Ano

Nessão sessão será retornado os valores de vendas em cada mês entre os anos de 2015-2019

In [ ]:
vendas_por_mes = {} # Local onde a quantidade de vendas por mês ficará armazenado

# Função pegar a soma de vendas por mês em cada ano
def soma_vendas_por_mes(valor):
    for x in range(5,10):
        ano = int(f'201{x}')
        df_ano_filtrado = df_copy.loc[df_copy['Ship_Date'].dt.year == ano]
        for m in range(1, 13):
            
            ultimo_dia = calendar.monthrange(ano, m)[1]
            data_inicial = datetime.strptime(f'201{x}-{m:02d}-01', '%Y-%m-%d')
            data_final = datetime.strptime(f'201{x}-{m:02d}-{ultimo_dia}', '%Y-%m-%d')
            
            valor_vendas_mes = round(df_ano_filtrado.loc[(df_ano_filtrado['Ship_Date'] >= data_inicial) & (df_ano_filtrado['Ship_Date'] <= data_final), 'Sales'].sum(), 2)
            
            if ano not in valor:
                valor.update({ano: {
                    f'mes_{m}':  f'$ {float(valor_vendas_mes)}'
                }})
            else:
               valor[ano].update({f'mes_{m}': f'$ {float(valor_vendas_mes)}'})
    return valor

soma_vendas_por_mes(vendas_por_mes)

Analisar vendas por Categoria e Sub-categoria de produtos

In [ ]:
# Criar o agrupamento por Categoria e Sub-Categoria para tirar a quantidade de vendas por produto (individual)
vendas_por_prod = df_copy.groupby(['Category', 'Sub_Category'])['Sales'].sum()
vendas_por_prod = pd.DataFrame(vendas_por_prod).sort_values(by='Sales', ascending=False)
vendas_por_prod.map("R$ {:,.2f}".format)

In [ ]:
# Criar um grupo com Categoria, para tirar a quantidade de vendas por categoria
vendas_por_categoria = df_copy.groupby('Category')['Sales'].sum()
vendas_por_categoria = pd.DataFrame(vendas_por_categoria).sort_values(by='Sales', ascending=False)
vendas_por_categoria.map("R$ {:,.2f}".format)

Analisar vendas por região

In [ ]:
vendas_por_regiao = df_copy.groupby('Region')['Sales'].sum()
vendas_por_regiao = pd.DataFrame(vendas_por_regiao)
vendas_por_regiao.map("R$ {:,.2f}".format).sort_values(by='Sales', ascending=False)

In [ ]:

vendas_por_regiao = df_copy.groupby(['Region', 'State'])['Sales'].sum()
vendas_por_regiao = pd.DataFrame(vendas_por_regiao).sort_values(by='Sales', ascending=False)
vendas_por_regiao.map("R$ {:,.2f}".format)

In [ ]:
qtd_vendas_prod = df_copy.groupby(['Category', 'Sub_Category'])['Sub_Category'].describe()
qtd_vendas_prod['count'].sort_values(ascending=False)

Analisar o comportamento do cliente, com números de compras, total gasto, qual os tipos de clientes que mais compram, e por qual região

In [ ]:
# Retorna a quantidade de vendas por tipo de cliente
tipo_cliente = df_copy.groupby('Segment').agg(
    Soma = ('Sales', 'sum'),
    Count = ('Sales', 'count'),
    Media = ('Sales', 'mean')
)

tipo_cliente['Soma']=tipo_cliente['Soma'].map("R$ {:,.2f}".format)
tipo_cliente['Media'] = tipo_cliente['Media'].map("R$ {:,.2f}".format)
tipo_cliente

In [ ]:
# Retorna a quantidade de compras por cliente, o faturamento total gerado pelas compras de cada cliente e a média de valor gasto por cliente
grupo = df_copy.groupby(['Segment', 'Customer_Name']).agg(
    qtd_compras = ('Sales', 'count'),
    faturamento = ('Sales', 'sum'),
    media = ('Sales', 'mean')
)
grupo['faturamento'] = grupo['faturamento'].map("R$ {:,.2f}".format)
grupo['media'] = grupo['media'].map("R$ {:,.2f}".format)
grupo.sort_values(by='qtd_compras', ascending=False).head(20)

In [ ]:
# Retorna a quantidade de compras por item específico, mostrando qual o cliente, e de qual segmento ele pertence
grupo2 = df_copy.groupby(['Segment','Customer_Name', 'Sub_Category']).agg(
    qtd_itens_cliente = ('Sub_Category', 'count'),
    compras = ('Sales', 'sum')
)
grupo2['compras'] = grupo2['compras'].map("R$ {:,.2f}".format)
grupo2.sort_values(by='qtd_itens_cliente', ascending=False).head(20)

Principais Insights da Análise

📊 A categoria de Tecnologia lidera as vendas, com cerca de R$ 100 mil a mais que as demais categorias. Esse desempenho reforça o potencial do setor e indica que investimentos adicionais em marketing, estoque e diversificação de modelos podem gerar retornos ainda maiores.

📊 Entre os produtos, os celulares apresentam o maior faturamento, consolidando o domínio da categoria de Tecnologia. No entanto, quando analisamos quantidade vendida, percebe-se uma presença mais forte de itens de escritório no top 20. Isso sugere que, embora produtos tecnológicos gerem mais receita por unidade, produtos básicos — de menor valor — têm maior volume e giro, sustentando parte importante das vendas.

📊 O segmento de clientes Consumer é o principal comprador, com vantagem próxima de R$ 500 mil em vendas sobre os demais segmentos. Esse resultado reforça que ações de marketing, promoções e campanhas de fidelização devem ser fortemente direcionadas ao público geral, que concentra a maior parte da receita.

📊 Em volume de vendas, fichários, papéis e móveis lideram o top 3, enquanto celulares aparecem logo depois. Isso evidencia que, apesar do alto faturamento dos celulares, sua frequência de compra é menor, enquanto produtos essenciais e de ticket baixo apresentam alta rotatividade, impulsionando a operação no dia a dia.

📊 Em termo de localidade, as regiões West e East se destacam liderando o ranking de faturamento, mostrando forte público nessa parte do páis, isso se nota também pelo fato dos estados com maior faturamento, constituídos de New York e a Califórnia, além de outros estados dessas regiões aparecendo repedidamente no topo do ranking

📊 Os estados de New York e Califórnia tem um ampla vantagem em relação aos demais abaixo deles, visto que do terceiro ao segundo (califórnia) tem uma grande diferença, que é aproximadamente o dobro, e do segundo ao primeiro é de aproximadamente R$ 150 mil.

### Conclusão

Em resumo a loja, apresenta um alto faturamento na categoria Tecnologia, porém, em volume e quantidade de vendas, mostra que os itens de escritórios de menor valor estão em grande maioria. Além disso, o público em geral (Consumer) mostra-se ser o motor de receita, com uma diferença de aproximandamente R$ 500 mil. Esses pontos em conjunto apresenta que, a estratégia ideal seria, fortalecer o portfólio tecnológico, mas reconhecendo que em giro rápido de vendas, produtos com menor valor, como os de escritórios são de alta importância para o lucro final, foi analisado que as regiões de West e East tem um grande faturamento com uma distância considerável em relação aos outros, cidade como New York e Califórnia são um bom alvo para a expansão e foco de vendas, visto o seu alto retorno.